# Introduction

A [monoid](https://en.wikipedia.org/wiki/Monoid) is a concept from [abstract algebra](https://en.wikipedia.org/wiki/Abstract_algebra) which represent sets with an identity element and an associative binary operation (which must be closed under the set). Herein, these two operations will be respectively denoted `mempty` and `mappend`, as in Haskell. 

It is also common to define an operation `mconcat` that reduces a list of monoid values into a single monoid value. Operation `mconcat` depends only on the previous operations and how the folding is executed.

The next abstract class includes an API for a monoid type:

In [108]:
from abc import ABC, abstractmethod

class Monoid(ABC):
  """ based on Christopher Harrison's «Dial M for Monoid» post """
  
  @staticmethod
  @abstractmethod
  def mempty():
    """ mempty :: m
        identity element of the monoid """
    ...

  @abstractmethod
  def mappend(self, rhs):
    """ mappend :: m -> m -> m (in Haskell also called <>)
        puts two monoids together """
    ...

  def __add__(self, rhs):
    """ operator '+' is mappend """
    return self.mappend(rhs)

  @classmethod
  def foldr(cls, fn, acc, lst):
    """ foldr :: (a->b->b) -> b -> [a] -> b
        reduce function, used by mconcat """
    if not lst:
      return cls.mempty()
    return fn(lst[0], cls.foldr(fn, acc, lst[1:]))

  @classmethod
  def mconcat(cls, xs):
    """ mconcat :: [m] -> m
        reduces a list of monoids into a single value """
    folder = lambda x, y: x.mappend(y)
    return cls.foldr(folder, cls.mempty(), xs)

A subclass of `Monoid` in order to define a valid monoid should follow the next rules (written in Haskell style):

+ `mempty <> a = a`
    
+ `a <> mempty = a`
    
+ `(a <> b) <> c = a <> (b <> c)`


## Sum Monoid

Let's consider, as a first example, the monoid $(0, +)$ over the integers:

In [3]:
class Sum(Monoid):
  def __init__(self, x):
    self.x = x

  @staticmethod
  def mempty():
    return Sum(0)

  def mappend(self, rhs):
    return Sum(self.x + rhs.x)

  def __repr__(self):
    return f'Sum({self.x})'

Let's make some tests if the monoid rules are holding:

In [123]:
# we didn't implement equality, so we're just printing both sides of each equation

zero = Sum.mempty()
print(zero+Sum(10), Sum(10)) # mempty <> a = a
print(Sum(10)+zero, Sum(10)) # a <> mempty = a

print((Sum(10)+Sum(20))+Sum(30), Sum(10)+(Sum(20)+Sum(30))) # (a <> b) <> c = a <> (b <> c)

Sum(10) Sum(10)
Sum(10) Sum(10)
Sum(60) Sum(60)


And some use cases:

In [116]:
Sum(13) + Sum(10) + zero + Sum(2000)

Sum(2023)

In [117]:
sums = [Sum(13), Sum(10), zero, Sum(2000)]
Sum.mconcat(sums)

Sum(2023)

## Vector Sum Monoid

Another example of monoid is $((0,0), \text{vector sum})$ over  $\mathbb{R}^2$.

In [6]:
class VectorSum(Monoid):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  @staticmethod
  def mempty():
    return VectorSum(0,0)

  def mappend(self, rhs):
    return VectorSum(self.x+rhs.x, self.y+rhs.y)

  def __repr__(self):
    return f'VectorSum({self.x},{self.y})'

In [7]:
null_vector = VectorSum.mempty()
VectorSum(3,4) + VectorSum(1,-1) + null_vector + VectorSum(-4,6)

VectorSum(0,9)

In [55]:
moves = [VectorSum(7,2), null_vector, VectorSum(-3,5), VectorSum(2,3)]
VectorSum.mconcat(moves)

VectorSum(6,10)

Let's now consider a more restricted example. Imagine a robot following moving orders given as vectors. However, if moving outside the first quadrant, the robot enters a failing state and would stay there no matter the remaining moves.

In [106]:
class Robot(VectorSum):
  def mappend(self, rhs):
    if self.x+rhs.x<0 or self.y+rhs.y<0:
      return Failed()
    return super().mappend(rhs)


class Failed(VectorSum):
  def __init__(self):
    pass

  def mappend(self, rhs):
    return self

  def __repr__(self):
    return 'Fail'

The append operation keep tabs on what is happening, no need for a more complex control structure:

In [107]:
stay = Robot.mempty()
print(Robot(3,4) + Robot( 4,-1) + stay + Robot(-4,6))
print(Robot(3,4) + Robot(-4,-1) + stay + Robot(-4,6))

VectorSum(3,9)
Fail


## Any and All Monoid

Another monoid is $(\text{False}, \lor)$ over the set $\mathbb{B}=\{\text{False},\text{True}\}$.

In [12]:
class Any(Monoid):
  def __init__(self, x):
    self.x = x

  @staticmethod
  def mempty():
    return Any(False)

  def mappend(self, rhs):
    return Any(self.x or rhs.x)

  def __call__(self, x):
    return Any(x)

  def __repr__(self):
    return f'Any({self.x})'

In [13]:
a = Any.mempty()
b = Any(True)
a + b + a + a

Any(True)

And monoid $(\text{True}, \land)$ over the set $\mathbb{B}=\{\text{False},\text{True}\}$.

In [67]:
class All(Monoid): 
  def __init__(self, x):
    self.x = x

  @staticmethod
  def mempty():
    return All(True)

  def mappend(self, rhs):
    return All(self.x and rhs.x)

  def __call__(self, x):
    return All(x)

  def __repr__(self):
    return f'All({self.x})'

In [118]:
a = All.mempty()
b = All(False)
a + b + a + a

All(False)

## The Endo and Function Monoid

A function-related monoid is $(\text{identity function}, \text{function composition})$ over the set of endomorphisms (functions of type `a->a`).

In [14]:
# instance Monoid (Endo a) where
#     mempty = Endo id
#     Endo f `mappend` Endo g = Endo (f . g)

class Endo(Monoid):
  def __init__(self, f):
    self.f = f

  @staticmethod
  def mempty():
    return Endo(lambda x: x)

  def mappend(self, rhs):
    return Endo(lambda x: self.f(rhs.f(x)))

  def __call__(self, x):
    return self.f(x)

Some use cases:

In [15]:
double = Endo(lambda x: 2*x)
succ = Endo(lambda x: x+1)

f = double + succ
f(100)

202

In [56]:
identity = Endo.mempty()
sq = Endo(lambda x: x**2)
fs = [sq, double, succ, identity, succ]

f = Endo.mconcat(fs)
f(100)

41616

In [17]:
prefix = Endo(lambda s: 'Hello '+s)
suffix = Endo(lambda s: s+"!")

(prefix + suffix)('World')

'Hello World!'

A generalization of `Endo` is a monoid about functions of type `a->b`. It's known that a function `a->b` is a monoid if `b` is a monoid.

Due to Python's limitations (or mine), a new monoid is needed for each given type `b`. The next one is about functions of type `a->All`.

In [119]:
# instance Monoid b => Monoid (a -> b) where
#     mempty _ = mempty
#     mappend f g x = f x `mappend` g x

class PredicateAll(Monoid):
  def __init__(self, f):
    self.f = lambda x: All(f(x))

  @staticmethod
  def mempty():
    empty = PredicateAll(lambda _: _) # dummy
    empty.f = lambda _: All.mempty()
    return empty

  def mappend(self, rhs):
    append = PredicateAll(lambda _: _) 
    append.f = lambda x: self.f(x) + rhs.f(x)
    return append

  def __call__(self, x):
    return self.f(x)

The next use case first defines a sequence of password rules:

In [120]:
# passwords must have more than seven chars
rule1 = lambda password: len(password) >= 8
# passwords must have at least one uppercase letter
rule2 = lambda password: any(c.isupper() for c in password)
# passwords must have at least one digit
rule3 = lambda password: any(c.isdigit() for c in password)
# passwords must have at least non-alphanumeric char
rule4 = lambda password: any(not c.isalnum() for c in password)

We can use the monoid `a->All` to easily combine different rules:

In [121]:
large_password = PredicateAll(rule1)
has_uppercase = PredicateAll(rule2)
rules = large_password + has_uppercase

print(rules('abderRTqd1'), rules('abdeSrd'), rules('12ABVda'))

All(True) All(False) All(False)


Using `map` and `mconcat` to join a sequence of rules:

In [122]:
rules = list(map(PredicateAll, [rule1, rule2, rule3, rule4]))
rules = PredicateAll.mconcat(rules)

print(rules('abderRTqd1'), rules('abdeSrd'), rules('12ABVda')) # invalid passwords
print(rules('abcDEF123%'))

All(False) All(False) All(False)
All(True)


Btw, to get the value outside of a monoid, we just need to include a getter method for that effect.

## List Monoid

In the next example, Christopher Harrison considers another monoid, $(\text{empty list}, \text{list concatenation})$ over the set of all lists. Herein, we define a new type List (not using Python's lists).

In [86]:
class List(Monoid):
  @staticmethod
  def mempty():
    return Nil()

  def mappend(self, rhs):
    return List.foldr(Cons, rhs, self)

  @classmethod
  def foldr(cls, fn, acc, lst):  # foldr was rewritten to deal with Cons(...) 
    if type(lst)==Nil:           # instead of Python lists
      return acc
    return fn(lst.head, cls.foldr(fn, acc, lst.tail))        


class Cons(List):
  def __init__(self, head, tail):
    self.head = head
    self.tail = tail

  def __repr__(self):
    if type(self.tail)==Nil:
      return repr(self.head)
    return repr(self.head) + ', ' + repr(self.tail)


class Nil(List):
  def __repr__(self):
    return ''

In [88]:
a = Cons(1, Cons(2, Cons(3, Nil())))
a

1, 2, 3

In [89]:
b = Cons(4, Cons(5, Cons(6, Nil())))
a + b

1, 2, 3, 4, 5, 6

In [90]:
c = Cons(7, Cons(8, Cons(9, Nil())))
d = List.mconcat(Cons(a, Cons(b, Cons(c, Nil()))))
d

1, 2, 3, 4, 5, 6, 7, 8, 9

In [91]:
List.foldr(lambda x,acc:x+acc, 0, d) # sum elements of list 'd'

45

In [93]:
List.foldr(lambda x,acc:f'{acc}{x}', '', d) # concatenates values into a string in reverse order

'987654321'

Harrison also shows how to prevent call stack increases, by producing a tail-call optimization using continuations and trampolines:

In [94]:
class List(Monoid):
  @staticmethod
  def mempty():
    return Nil()

  def mappend(self, rhs):
    return List.foldr(Cons, rhs, self)

  @classmethod
  def foldr(cls, fn, acc, lst, cont):
    if type(lst)==Nil:
      return cont(acc)
    return lambda: cls.foldr(fn, acc, lst.tail, lambda v: lambda: cont(fn(lst.head, v)))

In [95]:
def trampoline(thunk):
  while callable(thunk): 
    thunk = thunk()
  return thunk

In [101]:
lst = Nil()
for i in range(10_000):  # a list with 10k elements
  lst = Cons(i, lst)     # if recursed over, it would produce a Stack Overflow

trampoline(List.foldr(lambda x,acc:x+acc, 0, lst, lambda i:i))

49995000

# References

+ Christopher Harrison - Functional Python, [pt1](https://www.tweag.io/blog/2022-09-08-fp1-typopaedia-pythonica/), [pt2](https://www.tweag.io/blog/2023-01-19-fp2-dial-m-for-monoid/) 

+ Oliver Balfour - [Monoids in Haskell](https://blog.oliverbalfour.com/haskell/2020/08/02/monoids-in-haskell.html)

+ Wikipedia - [Monoid](https://en.wikipedia.org/wiki/Monoid)